# import

In [1]:
import os
import random
import glob
import re

import pandas as pd
import numpy as np

from sklearn.preprocessing import RobustScaler

import torch
import torch.nn as nn
from tqdm import tqdm

In [2]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager

font_path="c:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=font_path).get_name()
matplotlib.rc('font',family=font_name)

plt.rcParams['font.size'] = 13  # 기본 폰트 크기
plt.rcParams['axes.labelsize'] = 13  # x,y축 label 폰트 크기
plt.rcParams['xtick.labelsize'] = 13  # x축 눈금 폰트 크기
plt.rcParams['ytick.labelsize'] = 13  # y축 눈금 폰트 크기
plt.rcParams['legend.fontsize'] = 13  # 범례 폰트 크기
plt.rcParams['figure.titlesize'] = 15  # figure title 폰트 크기

import colorsys

def generate_colors(n):
    return [colorsys.hsv_to_rgb(i/n, 0.7, 0.9) for i in range(n)]

In [3]:
# GPU 이름 출력
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("GPU is not available, using CPU.")

GPU: NVIDIA GeForce RTX 4080 Laptop GPU


# constant

In [18]:
LOOKBACK = 28
PREDICT = 7
BATCH_SIZE = 32
EPOCHS = 100

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# def

In [5]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

In [19]:
import pandas as pd


# 성수기 구분 데이터 생성 함수
def generate_peak_season_data(df):
    # 업장+메뉴 단위로 월별 평균 매출 계산
    monthly_avg = (
        df.groupby(["영업장명", "메뉴명", "월"])["매출수량"]
        .mean()
        .reset_index()
    )

    # 각 조합별 평균 매출
    base_avg = (
        monthly_avg.groupby(["영업장명", "메뉴명"])["매출수량"]
        .mean()
        .reset_index()
        .rename(columns={"매출수량": "기준매출"})
    )

    # 병합 후 성수기/비수기 구분 (임계치: 평균 대비 1.3배 이상 = 성수기, 0.7배 이하 = 비수기)
    monthly_labeled = pd.merge(monthly_avg, base_avg, on=["영업장명", "메뉴명"])
    monthly_labeled["성수기여부"] = monthly_labeled.apply(
        lambda row: "성수기" if row["매출수량"] >= row["기준매출"] * 1.3
        else "비수기" if row["매출수량"] <= row["기준매출"] * 0.7
        else "구분 불가",
        axis=1
    )
    # 결과: 업장+메뉴+월별 성수기 정보
    return monthly_labeled[["영업장명", "메뉴명", "월", "성수기여부"]].copy()


def preprocess_sales_data_first(df):
    # 날짜 변환
    df["영업일자"] = pd.to_datetime(df["영업일자"])

    # 영업장명과 메뉴명 분리
    df["영업장명"] = df["영업장명_메뉴명"].apply(lambda x: x.split("_")[0])
    df["메뉴명"] = df["영업장명_메뉴명"].apply(lambda x: x.split("_")[1])

    # 요일 및 주말 여부
    df["월"] = df["영업일자"].dt.month
    df["요일"] = df["영업일자"].dt.weekday
    df["주말여부"] = df["요일"].isin([5, 6]).astype(int)

    return df


def preprocess_sales_data_second(df, peak_season_map=None):
    # 공휴일 하드코딩 (2023~2025)
    holidays_2023_2025 = pd.to_datetime([
        # 2023년
        "2023-01-01", "2023-01-21", "2023-01-22", "2023-01-23", "2023-03-01", "2023-05-05",
        "2023-05-27", "2023-06-06", "2023-08-15", "2023-09-28", "2023-09-29", "2023-09-30",
        "2023-10-03", "2023-10-09", "2023-12-25",
        # 2024년
        "2024-01-01", "2024-02-09", "2024-02-10", "2024-02-11", "2024-03-01", "2024-05-05",
        "2024-05-06", "2024-06-06", "2024-08-15", "2024-09-16", "2024-09-17", "2024-09-18",
        "2024-10-03", "2024-10-09", "2024-12-25",
        # 2025년
        "2025-01-01", "2025-01-28", "2025-01-29", "2025-01-30", "2025-03-01", "2025-05-05",
        "2025-06-06", "2025-08-15", "2025-10-03", "2025-10-06", "2025-10-07", "2025-10-08",
        "2025-10-09", "2025-12-25"
    ])
    df["공휴일여부"] = df["영업일자"].isin(holidays_2023_2025).astype(int)

    # 성수기 여부 판별
    # 월 컬럼 생성 후 성수기 맵과 병합
    df["월"] = df["영업일자"].dt.month
    df = df.merge(
        peak_season_map,
        how="left",
        on=["영업장명", "메뉴명", "월"]
    )

    # 성수기여부 결측값은 "구분 불가" 처리
    df["성수기여부"] = df["성수기여부"].fillna("구분 불가")

    # time_idx 추가 (시계열 인덱스)
    df = df.sort_values(["영업장명", "메뉴명", "영업일자"])
    df["time_idx"] = df.groupby(["영업장명", "메뉴명"]).cumcount()

    # group_id 추가
    df["group_id"] = df["영업장명"] + "_" + df["메뉴명"]

    return df


def advanced_feature_engineering(df):
    """고급 피처 엔지니어링"""
    df = df.copy()

    # 🔥 이동평균 피처
    for window in [3, 7, 14]:
        df[f'ma_{window}'] = df.groupby('영업장명_메뉴명')['매출수량'].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean()
        )

    # 🔥 lag 피처
    for lag in [1, 7, 14]:
        df[f'lag_{lag}'] = df.groupby('영업장명_메뉴명')['매출수량'].shift(lag)

    # 🔥 계절성 피처
    df['month'] = df['영업일자'].dt.month
    df['day_of_month'] = df['영업일자'].dt.day
    df['quarter'] = df['영업일자'].dt.quarter

    # 🔥 순환적 시간 피처 (sin/cos 변환)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['day_sin'] = np.sin(2 * np.pi * df['요일'] / 7)
    df['day_cos'] = np.cos(2 * np.pi * df['요일'] / 7)

    # 결측값 처리
    df = df.fillna(0)

    return df

In [20]:
class MultiOutputLSTM(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, num_layers=2, output_dim=7):
        super(MultiOutputLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])  # (B, output_dim)


class EnhancedLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=3, output_dim=7, dropout=0.2):
        super(EnhancedLSTM, self).__init__()

        # 🔥 개선: Dropout, Batch Normalization 추가
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers,
                            batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.batch_norm = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(dropout)

        # 🔥 개선: 다층 FC 네트워크
        self.fc1 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc2 = nn.Linear(hidden_dim // 2, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_output = lstm_out[:, -1, :]  # (batch, hidden_dim)

        # Batch normalization (2D 입력 필요)
        if last_output.size(0) > 1:  # batch_size > 1일 때만
            last_output = self.batch_norm(last_output)

        x = self.dropout(last_output)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

In [21]:
def train_lstm(train_df):
    trained_models = {}

    for store_menu, group in tqdm(train_df.groupby(['영업장명_메뉴명']), desc ='Training LSTM'):
        store_train = group.sort_values('영업일자').copy()
        if len(store_train) < LOOKBACK + PREDICT:
            continue

        features = ['매출수량']
        scaler = RobustScaler()

        # DataFrame이 아닌 numpy 배열로 일관되게 처리
        train_vals = store_train[features].values  # 먼저 numpy로 변환
        train_vals_scaled = scaler.fit_transform(train_vals)  # numpy로 fit
        # store_train[features] = scaler.fit_transform(store_train[features])
        # train_vals = store_train[features].values  # shape: (N, 1)

        # 시퀀스 구성
        X_train, y_train = [], []
        for i in range(len(train_vals_scaled) - LOOKBACK - PREDICT + 1):
            X_train.append(train_vals_scaled[i:i+LOOKBACK])
            y_train.append(train_vals_scaled[i+LOOKBACK:i+LOOKBACK+PREDICT, 0])

        # 리스트를 numpy 배열로 먼저 변환
        X_train = torch.tensor(np.array(X_train)).float().to(DEVICE)
        y_train = torch.tensor(np.array(y_train)).float().to(DEVICE)

        model = MultiOutputLSTM(input_dim=1, output_dim=PREDICT).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.MSELoss()

        model.train()
        for epoch in range(EPOCHS):
            idx = torch.randperm(len(X_train))
            for i in range(0, len(X_train), BATCH_SIZE):
                batch_idx = idx[i:i+BATCH_SIZE]
                X_batch, y_batch = X_train[batch_idx], y_train[batch_idx]
                output = model(X_batch)
                loss = criterion(output, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        trained_models[store_menu] = {
            'model': model.eval(),
            'scaler': scaler,
            'last_sequence': train_vals_scaled[-LOOKBACK:]  # (28, 1)
        }

    return trained_models


def train_lstm_improved(train_df):
    trained_models = {}

    for store_menu, group in tqdm(train_df.groupby(['영업장명_메뉴명']), desc='Training Enhanced LSTM'):
        store_train = group.sort_values('영업일자').copy()
        if len(store_train) < LOOKBACK + PREDICT:
            continue

        # 🔥 개선: 다중 피처 사용
        features = ['매출수량', '요일', '주말여부', '공휴일여부']

        # 성수기여부 원핫 인코딩
        season_dummies = pd.get_dummies(store_train['성수기여부'], prefix='성수기')
        feature_data = pd.concat([store_train[features], season_dummies], axis=1)

        scaler = RobustScaler()
        train_vals_scaled = scaler.fit_transform(feature_data.values)

        # 시퀀스 구성
        X_train, y_train = [], []
        for i in range(len(train_vals_scaled) - LOOKBACK - PREDICT + 1):
            X_train.append(train_vals_scaled[i:i+LOOKBACK])
            y_train.append(train_vals_scaled[i+LOOKBACK:i+LOOKBACK+PREDICT, 0])  # 매출수량만 예측

        X_train = torch.tensor(np.array(X_train)).float().to(DEVICE)
        y_train = torch.tensor(np.array(y_train)).float().to(DEVICE)

        # 🔥 개선: 입력 차원 수정
        input_dim = feature_data.shape[1]
        model = MultiOutputLSTM(input_dim=input_dim, hidden_dim=128, num_layers=3, output_dim=PREDICT).to(DEVICE)

        # ✅ 훈련 코드 추가 (누락된 부분)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.MSELoss()

        model.train()
        for epoch in range(EPOCHS):
            idx = torch.randperm(len(X_train))
            for i in range(0, len(X_train), BATCH_SIZE):
                batch_idx = idx[i:i+BATCH_SIZE]
                X_batch, y_batch = X_train[batch_idx], y_train[batch_idx]

                optimizer.zero_grad()
                output = model(X_batch)
                loss = criterion(output, y_batch)
                loss.backward()
                optimizer.step()

        trained_models[store_menu] = {
            'model': model.eval(),
            'scaler': scaler,
            'feature_columns': feature_data.columns.tolist(),
            'last_sequence': train_vals_scaled[-LOOKBACK:]
        }

    return trained_models


def train_lstm_with_validation(train_df):
    trained_models = {}

    for store_menu, group in tqdm(train_df.groupby(['영업장명_메뉴명']), desc='Training with Validation'):
        store_train = group.sort_values('영업일자').copy()
        if len(store_train) < LOOKBACK + PREDICT + 14:  # 검증용 데이터 확보
            continue

        # 🔥 개선: Train/Validation 분할
        val_size = 14  # 2주치 검증 데이터
        train_data = store_train[:-val_size]
        val_data = store_train[-val_size:]

        # 피처 준비
        features = ['매출수량', '요일', '주말여부', '공휴일여부']
        season_dummies = pd.get_dummies(store_train['성수기여부'], prefix='성수기')
        feature_data = pd.concat([store_train[features], season_dummies], axis=1)

        scaler = RobustScaler()
        all_scaled = scaler.fit_transform(feature_data.values)

        # 훈련/검증 데이터 분할
        train_scaled = all_scaled[:-val_size]
        val_scaled = all_scaled[-val_size:]

        # 시퀀스 생성
        X_train, y_train = create_sequences(train_scaled, LOOKBACK, PREDICT)
        X_val, y_val = create_sequences(val_scaled, LOOKBACK, PREDICT)

        if len(X_train) == 0 or len(X_val) == 0:
            continue

        # 모델 훈련
        model = EnhancedLSTM(input_dim=feature_data.shape[1],
                            hidden_dim=128, num_layers=3, output_dim=PREDICT).to(DEVICE)

        # 🔥 개선: 학습률 스케줄링, Early Stopping
        optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.5)
        criterion = nn.MSELoss()

        best_val_loss = float('inf')
        patience_counter = 0
        best_model_state = None

        for epoch in range(EPOCHS):
            # 훈련
            model.train()
            train_loss = 0
            for i in range(0, len(X_train), BATCH_SIZE):
                batch_X = X_train[i:i+BATCH_SIZE]
                batch_y = y_train[i:i+BATCH_SIZE]

                optimizer.zero_grad()
                output = model(batch_X)
                loss = criterion(output, batch_y)
                loss.backward()

                # 🔥 개선: Gradient Clipping
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

                optimizer.step()
                train_loss += loss.item()

            # 검증
            model.eval()
            with torch.no_grad():
                val_output = model(X_val)
                val_loss = criterion(val_output, y_val).item()

            scheduler.step(val_loss)

            # Early Stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                best_model_state = model.state_dict().copy()
            else:
                patience_counter += 1
                if patience_counter >= 20:  # Early stopping
                    break

        # 최고 성능 모델 복원
        if best_model_state is not None:
            model.load_state_dict(best_model_state)

        trained_models[store_menu] = {
            'model': model.eval(),
            'scaler': scaler,
            'feature_columns': feature_data.columns.tolist(),
            'last_sequence': all_scaled[-LOOKBACK:]
        }

    return trained_models

def create_sequences(data, lookback, predict):
    X, y = [], []
    for i in range(len(data) - lookback - predict + 1):
        X.append(data[i:i+lookback])
        y.append(data[i+lookback:i+lookback+predict, 0])  # 매출수량만 예측

    if len(X) > 0:
        return torch.tensor(np.array(X)).float().to(DEVICE), torch.tensor(np.array(y)).float().to(DEVICE)
    else:
        return torch.tensor([]).to(DEVICE), torch.tensor([]).to(DEVICE)

In [22]:
def predict_lstm(test_df, trained_models, test_prefix: str):
    results = []

    for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
        key = store_menu
        if key not in trained_models:
            continue

        model = trained_models[key]['model']
        scaler = trained_models[key]['scaler']

        store_test_sorted = store_test.sort_values('영업일자')
        recent_vals = store_test_sorted['매출수량'].values[-LOOKBACK:]
        if len(recent_vals) < LOOKBACK:
            continue

        # 정규화
        recent_vals = scaler.transform(recent_vals.reshape(-1, 1))
        x_input = torch.tensor([recent_vals]).float().to(DEVICE)

        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()

        # 역변환
        restored = []
        for i in range(PREDICT):
            dummy = np.zeros((1, 1))
            dummy[0, 0] = pred_scaled[i]
            restored_val = scaler.inverse_transform(dummy)[0, 0]
            restored.append(max(restored_val, 0))

        # 예측일자: TEST_00+1일 ~ TEST_00+7일
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(PREDICT)]

        for d, val in zip(pred_dates, restored):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu,
                '매출수량': val
            })

    return pd.DataFrame(results)


def predict_lstm_improved(test_df, trained_models, test_prefix: str):
    results = []

    for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
        if store_menu not in trained_models:
            continue

        model_info = trained_models[store_menu]
        model = model_info['model']
        scaler = model_info['scaler']
        feature_columns = model_info['feature_columns']

        store_test_sorted = store_test.sort_values('영업일자')

        # 🔥 개선: 모든 피처 사용
        # 피처 재구성 (훈련 시와 동일한 순서)
        features = ['매출수량', '요일', '주말여부', '공휴일여부']
        season_dummies = pd.get_dummies(store_test_sorted['성수기여부'], prefix='성수기')

        # 훈련 시와 동일한 컬럼 순서 맞추기
        for col in feature_columns:
            if col not in season_dummies.columns and col not in features:
                season_dummies[col] = 0

        feature_data = pd.concat([store_test_sorted[features], season_dummies[feature_columns[len(features):]]], axis=1)
        feature_data = feature_data[feature_columns]  # 컬럼 순서 맞추기

        if len(feature_data) < LOOKBACK:
            continue

        # 정규화
        recent_vals = scaler.transform(feature_data.values[-LOOKBACK:])
        x_input = torch.tensor([recent_vals]).float().to(DEVICE)

        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()

        # 역변환 (매출수량만)
        restored = []
        for i in range(PREDICT):
            dummy = np.zeros((1, len(feature_columns)))
            dummy[0, 0] = pred_scaled[i]  # 첫 번째가 매출수량
            restored_val = scaler.inverse_transform(dummy)[0, 0]
            restored.append(max(restored_val, 0))

        # 결과 저장
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(PREDICT)]
        for d, val in zip(pred_dates, restored):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu,
                '매출수량': val
            })

    return pd.DataFrame(results)

In [23]:
def convert_to_submission_format(pred_df: pd.DataFrame, sample_submission: pd.DataFrame):
    # (영업일자, 메뉴) → 매출수량 딕셔너리로 변환
    pred_dict = dict(zip(
        zip(pred_df['영업일자'], pred_df['영업장명_메뉴명']),
        pred_df['매출수량']
    ))

    final_df = sample_submission.copy()

    for row_idx in final_df.index:
        date = final_df.loc[row_idx, '영업일자']
        for col in final_df.columns[1:]:  # 메뉴명들
            final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)

    return final_df

In [24]:
def calculate_smape(actual, predicted):
    """SMAPE 계산 함수"""
    actual = np.array(actual)
    predicted = np.array(predicted)

    # 0으로 나누기 방지
    denominator = (np.abs(actual) + np.abs(predicted)) / 2
    denominator = np.where(denominator == 0, 1e-8, denominator)

    smape = np.abs(actual - predicted) / denominator
    return np.mean(smape) * 100

def evaluate_model_performance(train_df, trained_models, validation_days=7):
    """
    모델 성능 평가 함수
    Args:
        train_df: 전체 훈련 데이터
        trained_models: 훈련된 모델들
        validation_days: 검증용으로 사용할 마지막 며칠의 데이터
    """
    store_performances = []
    store_weights = {}

    print("모델 성능 평가 중...")

    for store_menu, group in tqdm(train_df.groupby(['영업장명_메뉴명']), desc='평가 진행'):
        if store_menu not in trained_models:
            continue

        store_data = group.sort_values('영업일자').copy()

        # 데이터가 충분하지 않은 경우 스킵
        if len(store_data) < LOOKBACK + validation_days:
            continue

        # 훈련 데이터와 검증 데이터 분할
        train_split = store_data[:-validation_days]
        validation_split = store_data[-validation_days:]

        # 실제 검증 데이터 값
        actual_values = validation_split['매출수량'].values

        # 예측 수행
        model = trained_models[store_menu]['model']
        scaler = trained_models[store_menu]['scaler']

        # 검증 시작점의 이전 LOOKBACK 일수 데이터 사용
        recent_vals = train_split['매출수량'].values[-LOOKBACK:]

        if len(recent_vals) < LOOKBACK:
            continue

        # 정규화
        recent_vals_scaled = scaler.transform(recent_vals.reshape(-1, 1))
        x_input = torch.tensor([recent_vals_scaled]).float().to(DEVICE)

        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()

        # 역변환
        predicted_values = []
        for i in range(min(PREDICT, validation_days)):
            dummy = np.zeros((1, 1))
            dummy[0, 0] = pred_scaled[i]
            restored_val = scaler.inverse_transform(dummy)[0, 0]
            predicted_values.append(max(restored_val, 0))

        # validation_days와 PREDICT 중 작은 값만큼만 평가
        eval_days = min(validation_days, PREDICT, len(actual_values))
        actual_eval = actual_values[:eval_days]
        predicted_eval = predicted_values[:eval_days]

        # SMAPE 계산
        smape = calculate_smape(actual_eval, predicted_eval)

        # 데이터 양에 따른 가중치 계산 (데이터가 적을수록 높은 가중치)
        zero_count = (store_data['매출수량'] == 0).sum()
        total_count = len(store_data)
        zero_ratio = zero_count / total_count

        # 0 비율이 높을수록 가중치가 높아지도록 설정
        # 최소 가중치를 1로, 최대 가중치를 10으로 제한
        weight = 1 / (1 - zero_ratio + 0.1)  # 0.1을 더해서 분모가 0이 되는 것을 방지
        weight = min(weight, 10)  # 최대 가중치 제한

        store_performances.append({
            '영업장명_메뉴명': store_menu,
            'SMAPE': smape,
            'Weight': weight,
            'Data_Count': total_count,
            'Zero_Count': zero_count,
            'Zero_Ratio': zero_ratio,
            'Weighted_SMAPE': smape * weight
        })

        store_weights[store_menu] = weight

    # 결과를 DataFrame으로 변환
    performance_df = pd.DataFrame(store_performances)

    if len(performance_df) == 0:
        print("평가할 수 있는 데이터가 없습니다.")
        return None, None

    # 전체 가중 평균 SMAPE 계산
    total_weighted_smape = performance_df['Weighted_SMAPE'].sum() / performance_df['Weight'].sum()

    # 통계 정보 출력
    print(f"\n=== 모델 성능 평가 결과 ===")
    print(f"평가 대상 업장 수: {len(performance_df)}")
    print(f"평균 SMAPE: {performance_df['SMAPE'].mean():.4f}")
    print(f"가중 평균 SMAPE: {total_weighted_smape:.4f}")
    print(f"SMAPE 표준편차: {performance_df['SMAPE'].std():.4f}")
    print(f"최고 성능 (lowest SMAPE): {performance_df['SMAPE'].min():.4f}")
    print(f"최악 성능 (highest SMAPE): {performance_df['SMAPE'].max():.4f}")
    print(f"평균 0 비율: {performance_df['Zero_Ratio'].mean():.4f}")
    print(f"평균 가중치: {performance_df['Weight'].mean():.4f}")

    # 상위/하위 성능 업장 출력
    print(f"\n=== 상위 10개 업장 (낮은 SMAPE) ===")
    top_performers = performance_df.nsmallest(10, 'SMAPE')
    for _, row in top_performers.iterrows():
        print(f"{row['영업장명_메뉴명']}: SMAPE={row['SMAPE']:.4f}, 0 비율={row['Zero_Ratio']:.4f}")

    print(f"\n=== 하위 10개 업장 (높은 SMAPE) ===")
    worst_performers = performance_df.nlargest(10, 'SMAPE')
    for _, row in worst_performers.iterrows():
        print(f"{row['영업장명_메뉴명']}: SMAPE={row['SMAPE']:.4f}, 0 비율={row['Zero_Ratio']:.4f}")

    return performance_df, total_weighted_smape

def evaluate_model_performance_improved(train_df, trained_models, validation_days=7):
    """
    개선된 모델 성능 평가 함수 (다중 피처 지원)
    """
    store_performances = []

    print("모델 성능 평가 중...")

    for store_menu, group in tqdm(train_df.groupby(['영업장명_메뉴명']), desc='평가 진행'):
        if store_menu not in trained_models:
            continue

        store_data = group.sort_values('영업일자').copy()

        # 데이터가 충분하지 않은 경우 스킵
        if len(store_data) < LOOKBACK + validation_days:
            continue

        # 훈련 데이터와 검증 데이터 분할
        train_split = store_data[:-validation_days]
        validation_split = store_data[-validation_days:]

        # 실제 검증 데이터 값
        actual_values = validation_split['매출수량'].values

        # 모델 정보 가져오기
        model_info = trained_models[store_menu]
        model = model_info['model']
        scaler = model_info['scaler']
        feature_columns = model_info['feature_columns']

        # ✅ 다중 피처로 예측 수행
        # 피처 재구성
        features = ['매출수량', '요일', '주말여부', '공휴일여부']
        season_dummies = pd.get_dummies(train_split['성수기여부'], prefix='성수기')

        # 훈련 시와 동일한 컬럼 순서 맞추기
        for col in feature_columns:
            if col not in season_dummies.columns and col not in features:
                season_dummies[col] = 0

        feature_data = pd.concat([train_split[features], season_dummies[feature_columns[len(features):]]], axis=1)
        feature_data = feature_data[feature_columns]  # 컬럼 순서 맞추기

        if len(feature_data) < LOOKBACK:
            continue

        # 정규화
        recent_vals = scaler.transform(feature_data.values[-LOOKBACK:])
        x_input = torch.tensor([recent_vals]).float().to(DEVICE)

        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()

        # 역변환
        predicted_values = []
        for i in range(min(PREDICT, validation_days)):
            dummy = np.zeros((1, len(feature_columns)))
            dummy[0, 0] = pred_scaled[i]  # 첫 번째가 매출수량
            restored_val = scaler.inverse_transform(dummy)[0, 0]
            predicted_values.append(max(restored_val, 0))

        # validation_days와 PREDICT 중 작은 값만큼만 평가
        eval_days = min(validation_days, PREDICT, len(actual_values))
        actual_eval = actual_values[:eval_days]
        predicted_eval = predicted_values[:eval_days]

        # SMAPE 계산
        smape = calculate_smape(actual_eval, predicted_eval)

        # 가중치 계산
        zero_count = (store_data['매출수량'] == 0).sum()
        total_count = len(store_data)
        zero_ratio = zero_count / total_count
        weight = 1 / (1 - zero_ratio + 0.1)
        weight = min(weight, 10)

        store_performances.append({
            '영업장명_메뉴명': store_menu,
            'SMAPE': smape,
            'Weight': weight,
            'Data_Count': total_count,
            'Zero_Count': zero_count,
            'Zero_Ratio': zero_ratio,
            'Weighted_SMAPE': smape * weight
        })

    # 결과를 DataFrame으로 변환
    performance_df = pd.DataFrame(store_performances)

    if len(performance_df) == 0:
        print("평가할 수 있는 데이터가 없습니다.")
        return None, None

    # 전체 가중 평균 SMAPE 계산
    total_weighted_smape = performance_df['Weighted_SMAPE'].sum() / performance_df['Weight'].sum()

    # 통계 정보 출력
    print(f"\n=== 모델 성능 평가 결과 ===")
    print(f"평가 대상 업장 수: {len(performance_df)}")
    print(f"평균 SMAPE: {performance_df['SMAPE'].mean():.4f}")
    print(f"가중 평균 SMAPE: {total_weighted_smape:.4f}")
    print(f"SMAPE 표준편차: {performance_df['SMAPE'].std():.4f}")
    print(f"최고 성능 (lowest SMAPE): {performance_df['SMAPE'].min():.4f}")
    print(f"최악 성능 (highest SMAPE): {performance_df['SMAPE'].max():.4f}")
    print(f"평균 0 비율: {performance_df['Zero_Ratio'].mean():.4f}")
    print(f"평균 가중치: {performance_df['Weight'].mean():.4f}")

    return performance_df, total_weighted_smape

def plot_performance_distribution(performance_df):
    """성능 분포 시각화 (선택사항)"""
    try:
        import matplotlib.pyplot as plt

        fig, axes = plt.subplots(2, 3, figsize=(18, 10))

        # SMAPE 분포
        axes[0, 0].hist(performance_df['SMAPE'], bins=30, alpha=0.7)
        axes[0, 0].set_title('SMAPE 분포')
        axes[0, 0].set_xlabel('SMAPE')
        axes[0, 0].set_ylabel('빈도')

        # 0 비율 vs SMAPE
        axes[0, 1].scatter(performance_df['Zero_Ratio'], performance_df['SMAPE'], alpha=0.6)
        axes[0, 1].set_title('0 비율 vs SMAPE')
        axes[0, 1].set_xlabel('0 비율')
        axes[0, 1].set_ylabel('SMAPE')

        # 0 비율 분포
        axes[0, 2].hist(performance_df['Zero_Ratio'], bins=30, alpha=0.7)
        axes[0, 2].set_title('0 비율 분포')
        axes[0, 2].set_xlabel('0 비율')
        axes[0, 2].set_ylabel('빈도')

        # 가중치 분포
        axes[1, 0].hist(performance_df['Weight'], bins=30, alpha=0.7)
        axes[1, 0].set_title('가중치 분포')
        axes[1, 0].set_xlabel('가중치')
        axes[1, 0].set_ylabel('빈도')

        # 0 비율 vs 가중치
        axes[1, 1].scatter(performance_df['Zero_Ratio'], performance_df['Weight'], alpha=0.6)
        axes[1, 1].set_title('0 비율 vs 가중치')
        axes[1, 1].set_xlabel('0 비율')
        axes[1, 1].set_ylabel('가중치')

        # 가중 SMAPE 분포
        axes[1, 1].hist(performance_df['Weighted_SMAPE'], bins=30, alpha=0.7)
        axes[1, 1].set_title('가중 SMAPE 분포')
        axes[1, 1].set_xlabel('가중 SMAPE')
        axes[1, 1].set_ylabel('빈도')

        plt.tight_layout()
        plt.show()

    except ImportError:
        print("matplotlib이 설치되지 않아 시각화를 건너뜁니다.")

# load

In [30]:
import pandas as pd

# 데이터 불러오기
train = pd.read_csv("data/train.csv")
train = preprocess_sales_data_first(train)
# 성수기 구분 데이터 생성
peak_season_map = generate_peak_season_data(train)
train = preprocess_sales_data_second(train, peak_season_map)

list_name = train["영업장명"].explode().unique()

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102676 entries, 0 to 102675
Data columns (total 12 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   영업일자      102676 non-null  datetime64[ns]
 1   영업장명_메뉴명  102676 non-null  object        
 2   매출수량      102676 non-null  int64         
 3   영업장명      102676 non-null  object        
 4   메뉴명       102676 non-null  object        
 5   월         102676 non-null  int32         
 6   요일        102676 non-null  int32         
 7   주말여부      102676 non-null  int64         
 8   공휴일여부     102676 non-null  int64         
 9   성수기여부     102676 non-null  object        
 10  time_idx  102676 non-null  int64         
 11  group_id  102676 non-null  object        
dtypes: datetime64[ns](1), int32(2), int64(4), object(5)
memory usage: 8.6+ MB


In [31]:
train[['영업장명_메뉴명', '영업일자', '영업장명', '메뉴명', '매출수량', '요일', '주말여부', '공휴일여부', '성수기여부']].head().T

,0,1,2,3,4
영업장명_메뉴명,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_1인 수저세트
영업일자,2023-01-01 00:00:00,2023-01-02 00:00:00,2023-01-03 00:00:00,2023-01-04 00:00:00,2023-01-05 00:00:00
영업장명,느티나무 셀프BBQ,느티나무 셀프BBQ,느티나무 셀프BBQ,느티나무 셀프BBQ,느티나무 셀프BBQ
메뉴명,1인 수저세트,1인 수저세트,1인 수저세트,1인 수저세트,1인 수저세트
매출수량,0,0,0,0,0
요일,6,0,1,2,3
주말여부,1,0,0,0,0
공휴일여부,1,0,0,0,0
성수기여부,비수기,비수기,비수기,비수기,비수기


In [33]:
train.to_csv("data/train_preprocessed.csv", index=False)

# train

In [29]:
# 학습
# trained_models = train_lstm_improved(train[['영업장명_메뉴명', '영업일자', '영업장명', '메뉴명', '매출수량', '요일', '주말여부', '공휴일여부', '성수기여부']])
trained_models = train_lstm_with_validation(train[['영업장명_메뉴명', '영업일자', '영업장명', '메뉴명', '매출수량', '요일', '주말여부', '공휴일여부', '성수기여부']])

Training with Validation:   0%|          | 0/193 [00:00<?, ?it/s]

Training with Validation: 100%|██████████| 193/193 [00:00<00:00, 243.91it/s]


# performance check

In [28]:
# 모델 학습 후
performance_df, weighted_smape = evaluate_model_performance_improved(train, trained_models)

# 시각화 (선택사항)
if performance_df is not None:
    plot_performance_distribution(performance_df)

모델 성능 평가 중...


평가 진행: 100%|██████████| 193/193 [00:00<00:00, 22281.27it/s]

평가할 수 있는 데이터가 없습니다.


# predict

In [ ]:
# all_preds = []

# # 모든 test_*.csv 순회
# test_files = sorted(glob.glob('data/TEST_*.csv'))

# for path in test_files:
#     test_df = pd.read_csv(path)
#     test_df = preprocess_sales_data(test_df)

#     # 파일명에서 접두어 추출 (예: TEST_00)
#     filename = os.path.basename(path)
#     test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

#     pred_df = predict_lstm(test_df, trained_models, test_prefix)
#     all_preds.append(pred_df)

# full_pred_df = pd.concat(all_preds, ignore_index=True)

: 

: 